In [1]:
import pandas as pd
import numpy as np

In [2]:
info_competition = pd.read_csv("./open_data/info_competition.csv", encoding = 'euc-kr')
info_login = pd.read_csv("./open_data/info_login.csv", encoding = 'euc-kr')
info_submission = pd.read_csv("./open_data/info_submission.csv", encoding = 'euc-kr')
info_user = pd.read_csv("./open_data/info_user.csv", encoding = 'euc-kr')

train = pd.read_csv("./open_data/train.csv", encoding = 'euc-kr')
train['DateTime'] = pd.to_datetime(train.DateTime)
train['date'] = train.DateTime.dt.date
train  = train.groupby('date').sum().reset_index()

C:\Users\82109\anaconda3\envs\pytorch_python3.7\lib\site-packages\IPython\core\interactiveshell.py:3147: DtypeWarning: Columns (4,5) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


## info_competition
- 일별 진행중인 대회 개수  + 대회 마감전 몇일 가중치
- 일별 전체 상금 규모

In [3]:
from datetime import timedelta

info_competition = info_competition[['period_start', 'period_end', 'prize']]
info_competition['period_start'] = pd.to_datetime(info_competition.period_start)
info_competition['period_start'] = info_competition.period_start.dt.date
info_competition['period_end'] = pd.to_datetime(info_competition.period_end)
info_competition['period_end'] = info_competition.period_end.dt.date

In [4]:
def daterange(d1, d2, prize):
    # 대회 종료 직전 3일은 1대신 1.5로 가중치
    return [[d1 + timedelta(days=i), prize, 1] if i <= (d2 - d1).days - 3 \
            else [d1 + timedelta(days=i), prize, 1.5] \
            for i in range((d2 - d1).days + 1)] 

df_competition = pd.DataFrame()
for i in range(info_competition.shape[0]):
    tmp = daterange(info_competition.period_start[i], info_competition.period_end[i], info_competition.prize[i]) # list
    df_tmp = pd.DataFrame(tmp, columns=['date', 'prize', 'num_competition'])
    df_competition = pd.concat([df_competition,df_tmp])
    
final_competition = df_competition.groupby('date').sum().reset_index()
train = pd.merge(train, final_competition, on='date', how='left')
train.tail()

,date,사용자,세션,신규방문자,페이지뷰,prize,num_competition
787,2020-11-04,4516,4472,1196,112683,9650.0,7.0
788,2020-11-05,4155,4037,1044,102901,9650.0,7.0
789,2020-11-06,3663,3576,825,88015,9650.0,7.0
790,2020-11-07,2472,2417,531,57386,9650.0,7.0
791,2020-11-08,2492,2420,522,50486,9650.0,7.0


## info_login
- train 데이터와 겹치므로 사용안함

## info_submission
- 날짜별 제출수만 사용

In [5]:
info_submission = pd.read_csv("./open_data/info_submission.csv", encoding = 'euc-kr')

info_submission = info_submission[['user_id', 'c_time']]
info_submission['c_time'] = pd.to_datetime(info_submission.c_time)
info_submission['c_time'] = info_submission.c_time.dt.date
info_submission = info_submission.groupby('c_time').count().reset_index()
info_submission.rename(columns = {'c_time' : 'date', 'user_id' : 'submission'}, inplace = True)
train = pd.merge(train, info_submission, on='date', how='left')
train.head()

,date,사용자,세션,신규방문자,페이지뷰,prize,num_competition,submission
0,2018-09-09,281,266,73,1826,850.0,1.0,71.0
1,2018-09-10,264,247,51,2092,850.0,1.0,74.0
2,2018-09-11,329,310,58,1998,850.0,1.5,94.0
3,2018-09-12,300,287,45,2595,850.0,1.5,93.0
4,2018-09-13,378,344,50,3845,850.0,1.5,94.0


## info_user
- 개별 유저의 데이터는 크게 영향을 끼치지 않을거라생각함
- 사용하지 않음


## 공휴일 / 주말

In [6]:
train['isHoliday'] = train['date'].apply(lambda x: 1 if x.weekday() == 5 or x.weekday() == 6 else 0)

In [7]:
import json
import requests
import urllib.parse as urlparse

def get_holidays(solYear, solMonth):
    def get_request_query(url, operation, params, serviceKey):
        params = urlparse.urlencode(params)
        request_query = url + '/' + operation + '?' + params + '&' + \
        'serviceKey' + '=' + serviceKey + '&' +'_type' + '=' + 'json'
        return request_query

    # 요청 URL과 오퍼레이션
    URL = 'http://apis.data.go.kr/B090041/openapi/service/SpcdeInfoService'
    OPERATION = 'getRestDeInfo' # 공휴일 정보 조회 오퍼레이션

    # 파라미터
    SERVICEKEY = '67yX%2BESCfbCn08mXGBSlxRdM5lqB3c%2BhCM%2Bhkte8ac%2BKT0rNHQ3Pd8OeuO8fzrbgrxcpCqhrUEbLorQGeOjfSw%3D%3D'
    PARAMS = {'solYear':solYear, 'solMonth':solMonth}

    request_query = get_request_query(URL, OPERATION, PARAMS, SERVICEKEY)
    response = requests.get(url=request_query)

    ret_dates = []
    holidays = {}
    if True == response.ok:
        holidays = json.loads(response.text)['response']['body']['items']
    if holidays:
        for i in holidays['item']:
            if i == 'dateKind':
                break
            ret_dates.append(i['locdate'])
        
    return ret_dates

In [8]:
month_range = pd.date_range(start='9/9/2018', end='11/8/2020', freq='M').strftime("%Y%m").tolist()
holidays = []
for i in month_range:
    holidays += get_holidays(str(i[:4]), str(i[4:]))

In [10]:
holiday_df = pd.DataFrame(holidays, columns=['date'])
holiday_df['date'] = pd.to_datetime(holiday_df.date, format='%Y%m%d')
aa = []
for i in train['date']:
    check = False
    for j in holiday_df['date']:
        if i == j:
            check = True
    if check == True:
        aa.append(1)
    else:
        aa.append(0)
train['cnt'] = aa

train['isHoliday'] = train['isHoliday'] + train['cnt']
train['isHoliday'] = train['isHoliday'].apply(lambda x: 1 if x == 2 else x)
del train['cnt']

## csv

In [11]:
train = train[['date', 'prize', 'num_competition', 'submission', 'isHoliday', '사용자', '세션', '신규방문자', '페이지뷰']]
train = train.fillna(0)
train.to_csv('./open_data/train.1.csv', index=False)## csv

In [15]:
pd.read_csv("./open_data/train.1.csv", encoding = 'utf-8')

,date,prize,num_competition,submission,isHoliday,사용자,세션,신규방문자,페이지뷰
0,2018-09-09,850.0,1.0,71.0,1,281,266,73,1826
1,2018-09-10,850.0,1.0,74.0,0,264,247,51,2092
2,2018-09-11,850.0,1.5,94.0,0,329,310,58,1998
3,2018-09-12,850.0,1.5,93.0,0,300,287,45,2595
4,2018-09-13,850.0,1.5,94.0,0,378,344,50,3845
...,...,...,...,...,...,...,...,...,...
787,2020-11-04,9650.0,7.0,434.0,0,4516,4472,1196,112683
788,2020-11-05,9650.0,7.0,511.0,0,4155,4037,1044,102901
789,2020-11-06,9650.0,7.0,383.0,0,3663,3576,825,88015
790,2020-11-07,9650.0,7.0,321.0,1,2472,2417,531,57386
